## Importing the Keras libraries and packages.

In [1]:
import tensorflow 
import keras
from keras.models import Sequential,Model
from keras.layers import Conv2D,Dense,Dropout,MaxPooling2D,Flatten,Input,ZeroPadding2D,LeakyReLU

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


## Checking the correct version.

In [0]:
# !pip uninstall  tensorflow
# !pip uninstall  keras

# !pip install  tensorflow==1.15
# !pip install  keras==2.1.6
#printing the versions
import keras as k
import tensorflow as tf
print(tf.__version__)
print(k.__version__)


## Download the keras_vggface weights.


In [0]:
!pip install keras_vggface

## Building the model.

In [0]:
#importing the model
from keras_vggface.vggface import VGGFace
from keras.utils.vis_utils import plot_model
 
#Specifie the width and height
img_width=240
img_height=300


#Initializing the model 
custom_vgg_model = VGGFace(weights="vggface", include_top=False,
	input_tensor=Input(shape=(img_width, img_height, 3)))
# custom_vgg_model.summary()


#Counting the layers
layer_count = 0
for layer in custom_vgg_model.layers:
	layer_count = layer_count+1

#Freezing the layers(except last 4 layers)
for l in range(layer_count-4):#-3 
	custom_vgg_model.layers[l].trainable=False
	# print(custom_vgg_model.layers[l])
	# print(l)

#Now,Except the last 4 layers all layers are freezed	
# custom_vgg_model.summary()

#Covert the layers to Sequential for training
model = Sequential()
for layer in custom_vgg_model.layers:
	# print(layer)
	model.add(layer)
# model.summary()

#Now flatten the layers
model.add(Flatten())
model.add(Dropout(0.3))
#Add Dense layers
model.add(Dense(units = 2056, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1056, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 256, activation = 'relu'))
#The last layer must contain 5 units(because we have 5 class to recognize)
model.add(Dense(units = 5, activation = 'softmax'))
# model.summary()
#Save the model architecture
plot_model(model, to_file='/content/drive/My Drive/Colab Notebooks/model_plot.png', show_shapes=True, show_layer_names=True)

## Compiling the model.

In [0]:
#Import the optimizer
from keras.optimizers import Adam
#the optimizer is initialized with learning rate
opt = Adam(lr=1e-4)
#Compile the model with loss function categorical_crossentropy and optimizer Adam and output the accuracy 
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

## Training the model.

In [0]:
#Initializing the variabels to respective directories(On Right side find the directory and right click copy path and past it)
train_data_dir = "/content/drive/My Drive/Colab Notebooks/dataset1/train"
val_data_dir   = "/content/drive/My Drive/Colab Notebooks/dataset1/val"
test_data_dir   = "/content/drive/My Drive/Colab Notebooks/dataset1/test"




batch_size=32
nb_epochs = 5
chfile = "/content/drive/My Drive/Colab Notebooks/faceacc3.hdf5"
#Checkpoints to get good results
#These checkpoints are monitored val_loss(it needs to be minimized),val_acc(it needs to be maximized),loss(it needs to be minimized),acc(it needs to be maximized)
checkpoints = [ModelCheckpoint(chfile,monitor = "val_loss",verbose=1,save_best_only=True,mode="min",period=1),
              ModelCheckpoint(chfile,monitor = "val_acc",verbose=1,save_best_only=True,mode="max",period=1),
              ModelCheckpoint(chfile,monitor = "loss",verbose=1,save_best_only=True,mode="min",period=1),
              ModelCheckpoint(chfile,monitor = "acc",verbose=1,save_best_only=True,mode="max",period=1)]



#Preaper the dataset for training
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   brightness_range=[0.3,2.0]
                                  )

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        classes=['aka','pra','rah','shr','unk'],
        batch_size=batch_size,
        class_mode='categorical')



#Preaper the dataset for validation
validation_datagen = ImageDataGenerator(rescale=1./255, 
                                   brightness_range=[0.3,1.3]
                                  )

validation_generator = validation_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        classes=['aka','pra','rah','shr','unk'],
        batch_size=batch_size,
         class_mode='categorical'
         )

# steps_per_epoch should be (number of training images total / batch_size) 
# validation_steps should be (number of validation images total / batch_size) 

history = model.fit_generator(
              train_generator,
              validation_data = validation_generator, 
              steps_per_epoch = train_generator.samples // batch_size,
              validation_steps = validation_generator.samples // batch_size,
              epochs = nb_epochs,
              callbacks=checkpoints)

## Ploting the model.

In [0]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc','val_acc','loss','val_loss'], loc='upper left')#,'tloss','vloss',
plt.show()


## Getting the test score(optinal).

In [0]:
#Prepare the dataset for testing
test_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range=[0.4,2.0])
test_generator = test_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        classes=['aka','pra','rah','shr','unk'],
        batch_size=batch_size)

#Get the score
score = model.evaluate_generator(validation_generator, 100)
print("Accuracy = ",score[1])

Found 50 images belonging to 5 classes.
Accuracy =  0.9737446669637081


## Saving the model.

In [0]:
#Save the model
model.save('/content/drive/My Drive/Colab Notebooks/modelname.hdf5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model


## Testing the model.

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os

#Load the model
model = load_model('/content/drive/My Drive/Colab Notebooks/faceacc3.hdf5')
# model.summary()

path ="path to image"
# /content/drive/My Drive/Colab Notebooks/dataset/test/a/ (1).jpg

test_image = image.load_img(path, target_size = (240,300))
test_image = image.img_to_array(test_image)
plt.imshow(test_image/255.)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

result1 = model.predict_proba(test_image/255)
predicted_classes = np.argmax(result1,axis=1)
print(result)


print('confidence : {:.3f}'.format(result1[0][predicted_classes[0]]))

if result[0][0]:
    print("aka")
elif result[0][1]:
    print("pra")
elif result[0][2]:
    print("rah")
elif result[0][3]:
    print("shy")
elif result[0][4]:
    print("unknown")


## Testing the entier test folder model
### In test folder the images should be numbered from 1 to 10(you can add more) for all students images.

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os

#Load the model
model = load_model('/content/drive/My Drive/Colab Notebooks/faceacc3.hdf5')
# model.summary()

s=["aka","pra","rah","shr","unk"]
# print(train_generator.class_indices)
for i in range(5):
  for j in range(1,11):
    path ="/content/drive/My Drive/Colab Notebooks/dataset1/test/"+s[i]+"/ ("+str(j)+").jpg"
    # /content/drive/My Drive/Colab Notebooks/dataset/test/a/ (1).jpg

    test_image = image.load_img(path, target_size = (240,300))
    test_image = image.img_to_array(test_image)
    plt.imshow(test_image/255.)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)

    result1 = model.predict_proba(test_image/255)
    predicted_classes = np.argmax(result1,axis=1)
    print(result)


    print('confidence : {:.3f}'.format(result1[0][predicted_classes[0]]))

    if result[0][0]:
        print("aka")
    elif result[0][1]:
        print("pra")
    elif result[0][2]:
        print("rah")
    elif result[0][3]:
        print("shy")
    elif result[0][4]:
        print("unknown")
